# Goal
Create a notebook that segments cat faces using SegNet

In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import theano
import sys

from skimage.io import imread
from matplotlib import pyplot as plt

import os
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS'] = 'mode=FAST_RUN, device=gpu0, floatX=float32, optimizer=fast_compile'

from keras import models
from keras.optimizers import SGD

# Load Data
Data comes from mom and I labeling, and google scraping using bin/scrape_google_images.py

![Cuter than a cat](https://s-media-cache-ak0.pinimg.com/originals/ea/7a/c1/ea7ac155ff8f9b19aa879ab656402b6a.jpg)

### Training: 
Pictures/train

### Testing:
Pictures/test

# Define Network

SegNet to start with: https://github.com/imlab-uiip/keras-segnet

In [17]:
path = 'Pictures/'
img_w = 256
img_h = 256
n_labels = 2

n_train = 189
n_test = 2

In [52]:
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
import json

img_w = 256
img_h = 256
n_labels = 2

kernel = 3

encoding_layers = [
    Conv2D(64, kernel, padding='same', input_shape=(n_train, img_h, img_w, 3)),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(128, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
]

autoencoder = models.Sequential()
autoencoder.encoding_layers = encoding_layers

for l in autoencoder.encoding_layers:
    autoencoder.add(l)

decoding_layers = [
    UpSampling2D(),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(128, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Conv2D(64, kernel, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(n_labels, 1, padding='valid'),
    BatchNormalization(),
]
autoencoder.decoding_layers = decoding_layers
for l in autoencoder.decoding_layers:
    autoencoder.add(l)

ValueError: Input 0 is incompatible with layer conv2d_53: expected ndim=4, found ndim=5

In [6]:
print(autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_27 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
activation_28 (Activation)   (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_28 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
activation_29 (Activation)   (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 64)      0         
__________

In [8]:
autoencoder.add(Reshape((n_labels, img_h * img_w)))
autoencoder.add(Permute((2, 1)))
autoencoder.add(Activation('softmax'))

with open('model_5l.json', 'w') as outfile:
    outfile.write(json.dumps(json.loads(autoencoder.to_json()), indent=2))
print('Compiled: OK')

Compiled: OK


In [47]:
def prep_data(path, mode):
    assert mode in {'test', 'train'}, \
        'mode should be either \'test\' or \'train\''
    data = []
    label = []
    df = pd.read_csv(path + mode + '.csv')
    n = n_train if mode == 'train' else n_test
    for i, item in df.iterrows():
        if i >= n:
            break
        img, gt = [imread(item[0])], np.clip(imread(item[1]), 0, 1)
        # imgn = img.reshape((img.shape[0], img.shape[1]*img.shape[2], img.shape[3], img.shape[4]))
        # print(imgn.shape)
        data.append(img)
        label.append(label_map(gt))
        sys.stdout.write('\r')
        sys.stdout.write(mode + ": [%-20s] %d%%" % ('=' * int(20. * (i + 1) / n - 1) + '>',
                                                    int(100. * (i + 1) / n)))
        sys.stdout.flush()
    sys.stdout.write('\r')
    sys.stdout.flush()
    
    data = np.array(data)
    data = data.reshape((data.shape[0], data.shape[1]*data.shape[2], data.shape[3], data.shape[4]))
    label = np.array(label).reshape((n, img_h * img_w, n_labels))

    print(mode + ': OK')
    print('\tshapes: {}, {}'.format(data.shape, label.shape))
    print('\ttypes:  {}, {}'.format(data.dtype, label.dtype))
    print('\tmemory: {}, {} MB'.format(data.nbytes / 1048576, label.nbytes / 1048576))

    return data, label

In [33]:
def label_map(labels):
    label_map = np.zeros([img_h, img_w, n_labels])    
    for r in range(img_h):
        for c in range(img_w):
            label_map[r, c, labels[r][c]] = 1
    return label_map

In [50]:
train_data, train_label = prep_data('../Pictures/', 'train')

train: OK==================>] 100%
	shapes: (189, 256, 256, 3), (189, 65536, 2)
	types:  uint8, float64
	memory: 35.4375, 189.0 MB


In [48]:
test_data, test_label = prep_data('../Pictures/', 'test')

test: OK==================>] 100%
	shapes: (2, 256, 256, 3), (2, 65536, 2)
	types:  uint8, float64
	memory: 0.375, 2.0 MB


In [51]:
nb_epoch = 50
batch_size = 18
autoencoder.compile(optimizer=SGD(lr = 0.1, decay = 1e-6, momentum = 0.9, nesterov = False),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
print(train_data.shape)
print(train_label.shape)

history = autoencoder.fit(train_data, train_label, batch_size=batch_size, epochs=nb_epoch, verbose=1)
autoencoder.save_weights('model_5l_weight_ep50.hdf5')

(189, 256, 256, 3)
(189, 65536, 2)


ValueError: Error when checking input: expected conv2d_27_input to have shape (None, 256, 256, 1) but got array with shape (189, 256, 256, 3)